In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
base_url = "https://rentahouse.com.ve/"

In [5]:
def get_links(base_url, page=1, casa=True):
    if casa:
        url = base_url + "casa.html"
        payload = { 'page' : page, 'orderBy' : 'entryTimestamp desc', 'propertyTypeFromSlug' : 'Casa' }
    else:
        url = base_url + "apartamento.html"
        payload = { 'page' : page, 'orderBy' : 'entryTimestamp desc', 'propertyTypeFromSlug' : 'Apartamento' }
    
    req = requests.get(url, params=payload)
    soup = BeautifulSoup(req.text, "html.parser")
    homes = soup.find_all("div", class_='property-list')

    res = []
    for home in homes:
        res.append(home.a['href'])
    
    return res

In [6]:
links = get_links(base_url)

In [7]:
def get_info_from_house(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")

    info = soup.find('div', class_="propertyInfo").find('div', class_="row")
    data = {}

    data['Price'] = info.find('div', class_='price')['title']

    general_info = info.find('ul', class_="property-detailes-list").find_all('li')

    for item in general_info:
        spans = item.find_all('span')
        data[spans[0].text] = spans[1].text

    location = info.find_all('div', class_='DescripcionGeneral')[-1].find_all('li')

    for item in location:
        spans = item.find_all('span')
        data[spans[0].text] = spans[1].text

    return data

In [8]:
get_info_from_house(links[0])

{'Price': '$498,000 USD',
 'Codígo RAH:': 'VE 23-26224',
 'Tipo de Propiedad:': 'Casa',
 'Estilo:': '1 Nivel',
 'Área Privada:': '948 m2\n\n',
 'Terreno:': '948 m2\n\n',
 'Estado Del Inmueble:': 'Usado',
 'Dormitorios:': '5',
 'Total Baños:': '8',
 'Baños Completos:': '8',
 'Tipo De Estacionamiento:': 'Cubierto',
 'Puestos De Estacionamiento:': '10',
 'Amoblado:': 'Si',
 '✅Dormitorio De Servicio:': 'Si',
 'Calle:': 'Publica',
 'País: ': 'Venezuela',
 'Estado: ': 'Distrito Metropolitano',
 'Ciudad: ': 'Caracas',
 'Urbanización: ': 'Lomas del Club Hipico'}

In [13]:
def get_house_links(pages, casa=True):
    homes = []
    for i in range(1,pages+1):
        print(f'Scrapeando pagina {i}')
        homes += get_links(base_url, page=i, casa=casa)
    
    return homes

In [10]:
house_links = get_house_links(30)

In [12]:
department_links = get_house_links(30, casa=False)

In [20]:
import sys
def scrape_homes(homes_list):
    homes = []
    count = 0
    for home in homes_list:
        homes.append(get_info_from_house(home))
        count += 1
        sys.stdout.write(f"\rElementos screapeados: {count}/{len(homes_list)}")

    return homes

In [21]:
houses = scrape_homes(house_links)

Elementos screapeados: 360/360

In [23]:
departments = scrape_homes(department_links)

Elementos screapeados: 360/360

In [25]:
import pandas as pd

In [45]:
houses_df = pd.DataFrame(houses)

In [47]:
houses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Price                        360 non-null    object
 1   Codígo RAH:                  360 non-null    object
 2   Tipo de Propiedad:           360 non-null    object
 3   Estilo:                      360 non-null    object
 4   Área Privada:                360 non-null    object
 5   Terreno:                     360 non-null    object
 6   Estado Del Inmueble:         360 non-null    object
 7   Dormitorios:                 360 non-null    object
 8   Total Baños:                 360 non-null    object
 9   Baños Completos:             360 non-null    object
 10  Tipo De Estacionamiento:     360 non-null    object
 11  Puestos De Estacionamiento:  306 non-null    object
 12  Amoblado:                    360 non-null    object
 13  ✅Dormitorio De Servicio:     149 no

In [57]:
def rename_columns(df):
    new_col_names = {}
    for col in df.columns:
        old_col = str(col)
        if old_col.endswith(':') or old_col.endswith(': '):
            new_col = old_col.split(':')[0]
            new_col_names[old_col] = new_col

    df.rename(columns=new_col_names, inplace=True)
    df.rename(columns = {'✅Dormitorio De Servicio' : 'Dormitorio De Servicio'}, inplace=True)


In [58]:
rename_columns(houses_df)

In [59]:
departments_df = pd.DataFrame(departments)
rename_columns(departments_df)
departments_df.columns

Index(['Price', 'Codígo RAH', 'Tipo de Propiedad', 'Estilo', 'Área Privada',
       'Estado Del Inmueble', 'Dormitorios', 'Total Baños', 'Baños Completos',
       'Tipo De Estacionamiento', 'Puestos De Estacionamiento', 'Amoblado',
       'Dormitorio De Servicio', 'País', 'Estado', 'Ciudad', 'Urbanización',
       'Medios Baños'],
      dtype='object')